In [3]:
# Import library yang dibutuhkan
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Konfigurasi parameter untuk model dan data
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_DIR = 'batik_jogja' # Pastikan folder ini ada di direktori yang sama
LEARNING_RATE = 0.001
EPOCHS = 10 # Anda bisa menambahkannya untuk akurasi yang lebih baik

# URL model EfficientNet-Lite0 dari TensorFlow Hub
HUB_MODEL_URL = "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2"

In [5]:
print("Memuat dataset dari direktori...")

# Dataset untuk training (80% dari data)
train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=123
)

# Dataset untuk validasi (20% dari data)
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=123
)

# Simpan nama kelas
class_names = np.array(train_dataset.class_names)
num_classes = len(class_names)
print("\nKelas yang ditemukan:", class_names)

Memuat dataset dari direktori...
Found 1350 files belonging to 6 classes.
Using 1080 files for training.
Found 1350 files belonging to 6 classes.
Using 270 files for validation.

Kelas yang ditemukan: ['batik_ceplok' 'batik_kawung' 'batik_nitik' 'batik_parang'
 'batik_sidoluhur' 'batik_truntum']


In [6]:
# Menggunakan cache dan prefetch untuk mempercepat loading data saat training
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

print("\nDataset siap digunakan.")


Dataset siap digunakan.


In [8]:
# Cell 5: Membangun Model (Lengkap)

print("Membangun model Keras dengan base model dari TensorFlow Hub...")

# Lapisan ini akan mengambil model EfficientNet-Lite0 dan tidak akan dilatih ulang (trainable=False)
feature_extractor_layer = hub.KerasLayer(
    HUB_MODEL_URL,
    trainable=False, 
    name='efficientnet_lite0_base',
    input_shape=IMG_SIZE + (3,)
)

# --- BLOK YANG ANDA GANTI (INI SUDAH BENAR) ---
# 1. Definisikan layer input secara eksplisit
inputs = tf.keras.Input(shape=IMG_SIZE + (3,))

# 2. Hubungkan input ke base model dari TensorFlow Hub
x = feature_extractor_layer(inputs)

# 3. Tambahkan lapisan Dropout setelahnya
x = tf.keras.layers.Dropout(0.2)(x)

# 4. Tambahkan lapisan output akhir
outputs = tf.keras.layers.Dense(num_classes, activation="softmax", name='output_layer')(x)

# 5. Buat model dengan mendefinisikan input dan outputnya
model = tf.keras.Model(inputs, outputs)
# --- AKHIR BLOK PENGGANTI ---

# Tampilkan ringkasan arsitektur model
model.summary()

Membangun model Keras dengan base model dari TensorFlow Hub...


TypeError: Exception encountered when calling layer 'efficientnet_lite0_base' (type KerasLayer).

Binding inputs to tf.function failed due to `A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
`. Received args: (<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, ragged=False, name=keras_tensor>,) and kwargs: {} for signature: (images: TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='images')) -> Dict[['default', TensorSpec(shape=(None, 1280), dtype=tf.float32,

In [ ]:
# Kompilasi model dengan optimizer, loss function, dan metrik
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Mulai proses pelatihan
print(f"\nMemulai pelatihan model untuk {EPOCHS} epoch...")
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS
)

In [ ]:
# Mengambil data histori pelatihan
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(EPOCHS)

# Membuat plot untuk Akurasi
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

# Membuat plot untuk Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Tampilkan plot
plt.show()

In [ ]:
# Simpan model yang sudah dilatih ke dalam file .h5
NEW_MODEL_PATH = 'batik_lite_model.h5'
print(f"\nMenyimpan model yang telah dilatih ke '{NEW_MODEL_PATH}'...")

model.save(NEW_MODEL_PATH)

print("Model baru berhasil disimpan!")